In [1]:
# ===== INSTALLATION =====
!pip install llama-parse llama-index-embeddings-huggingface
!pip install chromadb llama-index-vector-stores-chroma
!pip install sqlalchemy pandas


  Using cached chromadb-1.4.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.2 kB)
  Using cached llama_index_vector_stores_chroma-0.5.5-py3-none-any.whl.metadata (413 bytes)
  Using cached build-1.4.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached pybase64-1.4.3-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (8.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.9 M

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env into environment variables

secret_key = os.getenv("LLAMA_CLOUD_API_KEY")


In [3]:
# import os
# import re
# import uuid
# import pandas as pd
# from typing import List, Dict, Tuple
# from io import StringIO
# from sqlalchemy import create_engine, inspect, MetaData, Table
# import sqlite3
# import json

# from llama_parse import LlamaParse
# from llama_index.core import VectorStoreIndex, Settings, StorageContext, Document
# from llama_index.core.schema import TextNode
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# import chromadb
# from llama_index.vector_stores.chroma import ChromaVectorStore


# # ========================================
# # AUTOMATED CONTENT DETECTOR
# # ========================================
# class AutomatedContentDetector:
#     """
#     Automatically detects and separates text from tables
#     No manual intervention required
#     """

#     TABLE_PATTERN = r'\|.*\|[\s\S]*?\n\|[-:\s|]+\|[\s\S]*?(?=\n\n|\n#{1,6}\s|\Z)'

#     @staticmethod
#     def detect_and_segment(text: str) -> Dict:
#         """
#         Automatically segment document into text and tables
#         Returns: {'text': [...], 'tables': [...]}
#         """
#         tables = []
#         table_positions = []

#         # Find all tables
#         for match in re.finditer(AutomatedContentDetector.TABLE_PATTERN,
#                                   text, re.MULTILINE):
#             table_md = match.group(0).strip()
#             start_pos = match.start()
#             end_pos = match.end()

#             # Auto-detect table name from surrounding context
#             context_before = text[max(0, start_pos-300):start_pos]
#             table_name = AutomatedContentDetector._auto_name_table(
#                 context_before, table_md, len(tables)
#             )

#             tables.append({
#                 'content': table_md,
#                 'start': start_pos,
#                 'end': end_pos,
#                 'name': table_name,
#                 'context': context_before[-200:] if len(context_before) > 200 else context_before
#             })
#             table_positions.append((start_pos, end_pos))

#         # Extract text segments (non-table parts)
#         text_segments = []
#         last_pos = 0

#         for start, end in table_positions:
#             if start > last_pos:
#                 text_chunk = text[last_pos:start].strip()
#                 if text_chunk and len(text_chunk) > 50:  # Ignore tiny fragments
#                     text_segments.append({
#                         'content': text_chunk,
#                         'position': last_pos
#                     })
#             last_pos = end

#         # Remaining text after last table
#         if last_pos < len(text):
#             remaining = text[last_pos:].strip()
#             if remaining and len(remaining) > 50:
#                 text_segments.append({
#                     'content': remaining,
#                     'position': last_pos
#                 })

#         return {
#             'text_segments': text_segments,
#             'table_segments': tables,
#             'stats': {
#                 'text_count': len(text_segments),
#                 'table_count': len(tables)
#             }
#         }

#     @staticmethod
#     def _auto_name_table(context: str, table_content: str, index: int) -> str:
#         """
#         Automatically generate table name from context or content
#         """
#         # Look for explicit table references
#         patterns = [
#             r'[Tt]able\s+(\d+)[:\.]?\s*([A-Za-z0-9\s]+)',
#             r'[Tt]able\s+([A-Za-z0-9_\s]+)[:\.]',
#             r'([A-Z][A-Za-z\s]+)\s+[Tt]able',
#         ]

#         for pattern in patterns:
#             match = re.search(pattern, context)
#             if match:
#                 name = match.group(1) if len(match.groups()) == 1 else match.group(2)
#                 return re.sub(r'[^a-zA-Z0-9_]', '_', name.strip())[:50]

#         # Fallback: use first column header
#         try:
#             lines = table_content.split('\n')
#             if len(lines) > 0:
#                 first_line = lines[0]
#                 headers = [h.strip() for h in first_line.split('|') if h.strip()]
#                 if headers:
#                     return f"table_{headers[0][:30].replace(' ', '_').lower()}"
#         except:
#             pass

#         # Ultimate fallback
#         return f"auto_table_{index + 1}"


# # ========================================
# # AUTOMATED TABLE TO SQL CONVERTER
# # ========================================
# class AutoSQLTableCreator:
#     """
#     Automatically converts markdown tables to SQL
#     Dynamically creates schema based on detected structure
#     NO MANUAL COLUMN SPECIFICATION NEEDED
#     """

#     def __init__(self, db_path: str = "./automated_tables.db"):
#         self.db_path = db_path
#         self.engine = create_engine(f'sqlite:///{db_path}')
#         self.metadata = MetaData()
#         self.table_registry = {}  # Track created tables

#         # Create metadata tracking table
#         with self.engine.connect() as conn:
#             conn.execute("""
#             CREATE TABLE IF NOT EXISTS _table_metadata (
#                 id INTEGER PRIMARY KEY AUTOINCREMENT,
#                 table_name TEXT UNIQUE,
#                 original_name TEXT,
#                 num_rows INTEGER,
#                 num_columns INTEGER,
#                 column_names TEXT,
#                 context_text TEXT,
#                 created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
#             )
#             """)
#             conn.commit()

#     def markdown_to_dataframe(self, markdown_table: str) -> pd.DataFrame:
#         """
#         Automatically parse markdown table to DataFrame
#         Handles various formats automatically
#         """
#         try:
#             # Clean the markdown
#             lines = [l.strip() for l in markdown_table.split('\n') if l.strip()]

#             # Method 1: Try pandas read_csv with pipe delimiter
#             try:
#                 df = pd.read_csv(
#                     StringIO('\n'.join(lines)),
#                     sep='|',
#                     skipinitialspace=True,
#                     skip_blank_lines=True
#                 )

#                 # Remove empty first/last columns (markdown artifacts)
#                 df = df.loc[:, ~df.columns.str.match('^Unnamed')]
#                 df = df.dropna(axis=1, how='all')

#                 # Remove separator row (contains ---)
#                 if len(df) > 0:
#                     df = df[~df.astype(str).apply(
#                         lambda row: row.str.contains('---').any(), axis=1
#                     )]

#                 # Clean column names
#                 df.columns = [str(col).strip() for col in df.columns]

#                 # Clean data
#                 df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)

#                 return df.reset_index(drop=True)

#             except Exception as e:
#                 print(f"Method 1 failed: {e}, trying alternate parsing...")

#                 # Method 2: Manual parsing
#                 rows = []
#                 header = None

#                 for line in lines:
#                     cells = [c.strip() for c in line.split('|') if c.strip()]

#                     # Skip separator lines
#                     if all('-' in cell or ':' in cell for cell in cells):
#                         continue

#                     if header is None:
#                         header = cells
#                     else:
#                         rows.append(cells)

#                 if header and rows:
#                     df = pd.DataFrame(rows, columns=header)
#                     return df

#                 return None

#         except Exception as e:
#             print(f"❌ Failed to parse table: {e}")
#             return None

#     def auto_clean_column_names(self, df: pd.DataFrame) -> pd.DataFrame:
#         """
#         Automatically clean column names for SQL compatibility
#         """
#         new_columns = []
#         for col in df.columns:
#             # Remove special characters, spaces -> underscores
#             clean = re.sub(r'[^a-zA-Z0-9_]', '_', str(col))
#             clean = re.sub(r'_+', '_', clean)  # Multiple underscores -> single
#             clean = clean.strip('_').lower()

#             # Handle empty or numeric-only column names
#             if not clean or clean.isdigit():
#                 clean = f'col_{len(new_columns)}'

#             # Ensure uniqueness
#             base_clean = clean
#             counter = 1
#             while clean in new_columns:
#                 clean = f"{base_clean}_{counter}"
#                 counter += 1

#             new_columns.append(clean)

#         df.columns = new_columns
#         return df

#     def auto_create_sql_table(
#         self,
#         df: pd.DataFrame,
#         table_name: str,
#         context: str = ""
#     ) -> Tuple[bool, str]:
#         """
#         Automatically create SQL table from DataFrame
#         Schema is inferred from DataFrame dtypes
#         Returns: (success: bool, actual_table_name: str)
#         """
#         if df is None or df.empty:
#             return False, None

#         try:
#             # Clean DataFrame
#             df = self.auto_clean_column_names(df)

#             # Sanitize table name for SQL
#             safe_table_name = re.sub(r'[^a-zA-Z0-9_]', '_', table_name)
#             safe_table_name = safe_table_name[:64].lower()  # SQL name length limit

#             # Make unique if exists
#             base_name = safe_table_name
#             counter = 1
#             while safe_table_name in self.table_registry:
#                 safe_table_name = f"{base_name}_{counter}"
#                 counter += 1

#             # ⭐ AUTOMATIC SCHEMA CREATION - pandas does it all!
#             df.to_sql(
#                 safe_table_name,
#                 self.engine,
#                 if_exists='replace',
#                 index=False,
#                 dtype=None  # Auto-infer types from DataFrame
#             )

#             # Store metadata
#             self.table_registry[safe_table_name] = {
#                 'original_name': table_name,
#                 'rows': len(df),
#                 'columns': list(df.columns),
#                 'context': context
#             }

#             # Save to metadata table
#             with self.engine.connect() as conn:
#                 conn.execute(f"""
#                 INSERT OR REPLACE INTO _table_metadata
#                 (table_name, original_name, num_rows, num_columns,
#                  column_names, context_text)
#                 VALUES (?, ?, ?, ?, ?, ?)
#                 """, (
#                     safe_table_name,
#                     table_name,
#                     len(df),
#                     len(df.columns),
#                     json.dumps(list(df.columns)),
#                     context[:1000]
#                 ))
#                 conn.commit()

#             print(f"✅ Auto-created SQL table: '{safe_table_name}'")
#             print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
#             print(f"   Schema: {list(df.columns)}")

#             return True, safe_table_name

#         except Exception as e:
#             print(f"❌ Failed to create table '{table_name}': {e}")
#             return False, None

#     def get_table_info(self, table_name: str) -> Dict:
#         """Get information about a created table"""
#         return self.table_registry.get(table_name, {})

#     def query_table(self, table_name: str, limit: int = 5) -> pd.DataFrame:
#         """Query any created table"""
#         try:
#             return pd.read_sql(
#                 f"SELECT * FROM {table_name} LIMIT {limit}",
#                 self.engine
#             )
#         except Exception as e:
#             print(f"Query failed: {e}")
#             return None


# # ========================================
# # AUTOMATED NODE CREATOR WITH LINKING
# # ========================================
# class AutomatedNodeCreator:
#     """
#     Automatically creates nodes from text and table segments
#     Maintains links between text and SQL tables
#     """

#     def __init__(self, sql_creator: AutoSQLTableCreator):
#         self.sql_creator = sql_creator

#     def create_nodes_from_segments(
#         self,
#         text_segments: List[Dict],
#         table_segments: List[Dict],
#         doc_id: str = None
#     ) -> List[TextNode]:
#         """
#         Automatically create all nodes with proper linking
#         """
#         nodes = []
#         doc_id = doc_id or str(uuid.uuid4())

#         # Create text nodes
#         for i, text_seg in enumerate(text_segments):
#             # Auto-detect if text mentions tables
#             table_refs = self._detect_table_references(
#                 text_seg['content'],
#                 table_segments
#             )

#             node = TextNode(
#                 text=text_seg['content'],
#                 id_=f"{doc_id}_text_{i}",
#                 metadata={
#                     'type': 'text',
#                     'doc_id': doc_id,
#                     'position': text_seg['position'],
#                     'has_table_refs': len(table_refs) > 0,
#                     'referenced_tables': table_refs,
#                     'sql_database': self.sql_creator.db_path
#                 }
#             )
#             nodes.append(node)

#         # Create table reference nodes
#         for i, table_seg in enumerate(table_segments):
#             # Get the actual SQL table name created
#             table_info = self.sql_creator.get_table_info(table_seg['name'])

#             if table_info:
#                 # Create searchable description of the table
#                 description = self._generate_table_description(
#                     table_seg['name'],
#                     table_info,
#                     table_seg['context']
#                 )

#                 node = TextNode(
#                     text=description,
#                     id_=f"{doc_id}_table_{i}",
#                     metadata={
#                         'type': 'table_reference',
#                         'doc_id': doc_id,
#                         'sql_table_name': table_seg['name'],
#                         'sql_database': self.sql_creator.db_path,
#                         'num_rows': table_info['rows'],
#                         'num_columns': table_info['columns'],
#                         'column_names': table_info['columns'],
#                         'context': table_seg['context'][:500]
#                     }
#                 )
#                 nodes.append(node)

#         return nodes

#     def _detect_table_references(
#         self,
#         text: str,
#         table_segments: List[Dict]
#     ) -> List[str]:
#         """
#         Auto-detect if text references any tables
#         """
#         refs = []
#         text_lower = text.lower()

#         for table_seg in table_segments:
#             # Check if table name or similar words appear in text
#             table_name_parts = table_seg['name'].lower().split('_')

#             # If any significant part of table name is in text
#             for part in table_name_parts:
#                 if len(part) > 3 and part in text_lower:
#                     refs.append(table_seg['name'])
#                     break

#         return list(set(refs))

#     def _generate_table_description(
#         self,
#         table_name: str,
#         table_info: Dict,
#         context: str
#     ) -> str:
#         """
#         Auto-generate searchable text description for table
#         """
#         description = f"""Table: {table_name}

# Context: {context}

# This table contains {table_info['rows']} rows of structured data with {table_info['columns']} columns.

# Columns: {', '.join(table_info['columns'])}

# The data in this table can be queried using SQL from the database: {self.sql_creator.db_path}
# Table name for queries: {table_name}

# This table was automatically extracted and structured from the document.
# """
#         return description


# # ========================================
# # COMPLETE AUTOMATED PIPELINE
# # ========================================
# class FullyAutomatedRAGPipeline:
#     """
#     ZERO MANUAL INTERVENTION PIPELINE
#     PDF → Automatic Text/Table Separation → Vector Store + SQL
#     """

#     def __init__(
#         self,
#         llamaparse_api_key: str,
#         vector_db_path: str = "./auto_vector_db",
#         sql_db_path: str = "./auto_tables.db"
#     ):
#         # Initialize components
#         self.parser = LlamaParse(
#             api_key=llamaparse_api_key,
#             result_type="markdown",
#             verbose=True
#         )

#         self.content_detector = AutomatedContentDetector()
#         self.sql_creator = AutoSQLTableCreator(sql_db_path)
#         self.node_creator = AutomatedNodeCreator(self.sql_creator)

#         # Setup embedding model (FREE)
#         Settings.embed_model = HuggingFaceEmbedding(
#             model_name="BAAI/bge-small-en-v1.5"
#         )

#         # Setup vector store
#         self.chroma_client = chromadb.PersistentClient(path=vector_db_path)
#         self.collection = self.chroma_client.get_or_create_collection(
#             "automated_rag"
#         )
#         self.vector_store = ChromaVectorStore(chroma_collection=self.collection)
#         self.storage_context = StorageContext.from_defaults(
#             vector_store=self.vector_store
#         )

#         # Track processed documents
#         self.processed_docs = []
#         self.all_nodes = []

#     def process_pdf(self, pdf_path: str) -> Dict:
#         """
#         MAIN AUTOMATED PIPELINE
#         Input: PDF path
#         Output: Complete RAG system (Vector Store + SQL)
#         """
#         print(f"\n{'='*70}")
#         print(f"🚀 AUTOMATED PROCESSING: {pdf_path}")
#         print(f"{'='*70}\n")

#         # Step 1: Parse PDF
#         print("📄 Step 1: Parsing PDF with LlamaParse...")
#         documents = self.parser.load_data(pdf_path)
#         print(f"   ✅ Parsed {len(documents)} document(s)")

#         total_stats = {
#             'text_segments': 0,
#             'table_segments': 0,
#             'nodes_created': 0,
#             'sql_tables_created': 0
#         }

#         # Process each document
#         for doc_idx, doc in enumerate(documents):
#             print(f"\n📑 Processing Document {doc_idx + 1}/{len(documents)}...")

#             # Step 2: Automatic content detection
#             print("   🔍 Step 2: Detecting text and tables automatically...")
#             segments = self.content_detector.detect_and_segment(doc.text)

#             print(f"      Text segments: {segments['stats']['text_count']}")
#             print(f"      Table segments: {segments['stats']['table_count']}")

#             # Step 3: Process tables → SQL
#             print("   🗄️  Step 3: Creating SQL tables automatically...")
#             for table_seg in segments['table_segments']:
#                 df = self.sql_creator.markdown_to_dataframe(table_seg['content'])

#                 if df is not None and not df.empty:
#                     success, sql_name = self.sql_creator.auto_create_sql_table(
#                         df,
#                         table_seg['name'],
#                         table_seg['context']
#                     )

#                     if success:
#                         table_seg['name'] = sql_name  # Update with actual SQL name
#                         total_stats['sql_tables_created'] += 1

#             # Step 4: Create nodes with linking
#             print("   🔗 Step 4: Creating nodes with automatic linking...")
#             nodes = self.node_creator.create_nodes_from_segments(
#                 segments['text_segments'],
#                 segments['table_segments'],
#                 doc_id=f"doc_{doc_idx}"
#             )

#             self.all_nodes.extend(nodes)
#             total_stats['nodes_created'] += len(nodes)
#             total_stats['text_segments'] += segments['stats']['text_count']
#             total_stats['table_segments'] += segments['stats']['table_count']

#             print(f"      ✅ Created {len(nodes)} nodes")

#         # Step 5: Build vector index
#         print(f"\n📊 Step 5: Building vector index...")
#         self.index = VectorStoreIndex(
#             nodes=self.all_nodes,
#             storage_context=self.storage_context,
#             show_progress=True
#         )

#         print(f"\n{'='*70}")
#         print(f"✅ AUTOMATED PIPELINE COMPLETE!")
#         print(f"{'='*70}")
#         print(f"📊 Statistics:")
#         print(f"   Text segments processed: {total_stats['text_segments']}")
#         print(f"   Tables detected: {total_stats['table_segments']}")
#         print(f"   SQL tables created: {total_stats['sql_tables_created']}")
#         print(f"   Vector nodes created: {total_stats['nodes_created']}")
#         print(f"\n💾 Storage:")
#         print(f"   Vector DB: {self.chroma_client._settings.persist_directory}")
#         print(f"   SQL DB: {self.sql_creator.db_path}")

#         return total_stats

#     def query(self, question: str, top_k: int = 5) -> Dict:
#         """
#         Query the automated RAG system
#         Automatically determines if SQL query is needed
#         """
#         print(f"\n{'='*70}")
#         print(f"❓ Query: {question}")
#         print(f"{'='*70}\n")

#         # Retrieve from vector store
#         retriever = self.index.as_retriever(similarity_top_k=top_k)
#         results = retriever.retrieve(question)

#         response = {
#             'question': question,
#             'vector_results': [],
#             'sql_data': []
#         }

#         print(f"📄 Found {len(results)} relevant segments:\n")

#         for i, result in enumerate(results):
#             print(f"--- Result {i+1} (Score: {result.score:.4f}) ---")
#             print(f"Type: {result.metadata['type']}")
#             print(f"Content: {result.text[:200]}...\n")

#             response['vector_results'].append({
#                 'score': result.score,
#                 'type': result.metadata['type'],
#                 'content': result.text[:500]
#             })

#             # If result references a SQL table, fetch data
#             if result.metadata['type'] == 'table_reference':
#                 table_name = result.metadata['sql_table_name']
#                 print(f"🗄️  SQL Table Referenced: {table_name}")

#                 table_data = self.sql_creator.query_table(table_name, limit=10)
#                 if table_data is not None:
#                     print(f"   Preview (first 5 rows):")
#                     print(table_data.head().to_string())
#                     print()

#                     response['sql_data'].append({
#                         'table_name': table_name,
#                         'data': table_data.to_dict('records')
#                     })

#             elif result.metadata.get('referenced_tables'):
#                 print(f"🔗 References tables: {result.metadata['referenced_tables']}\n")

#         return response

#     def list_all_tables(self):
#         """List all automatically created tables"""
#         print(f"\n{'='*70}")
#         print("📋 ALL CREATED SQL TABLES")
#         print(f"{'='*70}\n")

#         for table_name, info in self.sql_creator.table_registry.items():
#             print(f"Table: {table_name}")
#             print(f"  Original Name: {info['original_name']}")
#             print(f"  Rows: {info['rows']}")
#             print(f"  Columns ({len(info['columns'])}): {', '.join(info['columns'][:5])}...")
#             print()


# # ========================================
# # USAGE - COMPLETELY AUTOMATED
# # ========================================

# # Initialize automated pipeline
# pipeline = FullyAutomatedRAGPipeline(
#     llamaparse_api_key=secret_key,  # "your-llamaparse-key",
#     vector_db_path="./ifct_vector_db",
#     sql_db_path="./ifct_tables.db"
# )

# # Process PDF - ZERO MANUAL WORK!
# stats = pipeline.process_pdf("IFCT2017.pdf")

# # Query the system
# response = pipeline.query("What is the protein content of rice?")

# # List all created tables
# pipeline.list_all_tables()

# # Access SQL directly if needed
# table_data = pipeline.sql_creator.query_table("table_proximate_principles", limit=20)
# print(table_data)


ObjectNotExecutableError: Not an executable object: '\n            CREATE TABLE IF NOT EXISTS _table_metadata (\n                id INTEGER PRIMARY KEY AUTOINCREMENT,\n                table_name TEXT UNIQUE,\n                original_name TEXT,\n                num_rows INTEGER,\n                num_columns INTEGER,\n                column_names TEXT,\n                context_text TEXT,\n                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n            )\n            '

In [4]:
# # ========================================
# # SECTION 1: INSTALLATION
# # ========================================
# # Run this first in Colab
# !pip install llama-parse llama-index llama-index-embeddings-huggingface
# !pip install chromadb llama-index-vector-stores-chroma
# !pip install sentence-transformers pandas



In [5]:
# ========================================
# SECTION 2: IMPORTS
# ========================================
import os
import re
import uuid
import json
import sqlite3
import pandas as pd
from typing import List, Dict, Tuple
from io import StringIO

from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, Settings, StorageContext
from llama_index.core.schema import TextNode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore


# ========================================
# SECTION 3: CONTENT DETECTOR CLASS
# ========================================
class AutomatedContentDetector:
    """
    Automatically detects and separates text from tables
    No manual intervention required
    """

    TABLE_PATTERN = r'\|.*\|[\s\S]*?\n\|[-:\s|]+\|[\s\S]*?(?=\n\n|\n#{1,6}\s|\Z)'

    @staticmethod
    def detect_and_segment(text: str) -> Dict:
        """
        Automatically segment document into text and tables
        Returns: {'text_segments': [...], 'table_segments': [...]}
        """
        tables = []
        table_positions = []

        # Find all tables
        for match in re.finditer(AutomatedContentDetector.TABLE_PATTERN,
                                  text, re.MULTILINE):
            table_md = match.group(0).strip()
            start_pos = match.start()
            end_pos = match.end()

            # Auto-detect table name from surrounding context
            context_before = text[max(0, start_pos-300):start_pos]
            table_name = AutomatedContentDetector._auto_name_table(
                context_before, table_md, len(tables)
            )

            tables.append({
                'content': table_md,
                'start': start_pos,
                'end': end_pos,
                'name': table_name,
                'context': context_before[-200:] if len(context_before) > 200 else context_before
            })
            table_positions.append((start_pos, end_pos))

        # Extract text segments (non-table parts)
        text_segments = []
        last_pos = 0

        for start, end in table_positions:
            if start > last_pos:
                text_chunk = text[last_pos:start].strip()
                if text_chunk and len(text_chunk) > 50:  # Ignore tiny fragments
                    text_segments.append({
                        'content': text_chunk,
                        'position': last_pos
                    })
            last_pos = end

        # Remaining text after last table
        if last_pos < len(text):
            remaining = text[last_pos:].strip()
            if remaining and len(remaining) > 50:
                text_segments.append({
                    'content': remaining,
                    'position': last_pos
                })

        return {
            'text_segments': text_segments,
            'table_segments': tables,
            'stats': {
                'text_count': len(text_segments),
                'table_count': len(tables)
            }
        }

    @staticmethod
    def _auto_name_table(context: str, table_content: str, index: int) -> str:
        """
        Automatically generate table name from context or content
        """
        # Look for explicit table references
        patterns = [
            r'[Tt]able\s+(\d+)[:\.]?\s*([A-Za-z0-9\s]+)',
            r'[Tt]able\s+([A-Za-z0-9_\s]+)[:\.]',
            r'([A-Z][A-Za-z\s]+)\s+[Tt]able',
        ]

        for pattern in patterns:
            match = re.search(pattern, context)
            if match:
                name = match.group(1) if len(match.groups()) == 1 else match.group(2)
                return re.sub(r'[^a-zA-Z0-9_]', '_', name.strip())[:50]

        # Fallback: use first column header
        try:
            lines = table_content.split('\n')
            if len(lines) > 0:
                first_line = lines[0]
                headers = [h.strip() for h in first_line.split('|') if h.strip()]
                if headers:
                    return f"table_{headers[0][:30].replace(' ', '_').lower()}"
        except:
            pass

        # Ultimate fallback
        return f"auto_table_{index + 1}"


# ========================================
# SECTION 4: SQL TABLE CREATOR CLASS (SIMPLE VERSION)
# ========================================
class AutoSQLTableCreatorSimple:
    """
    Pure SQLite version - No SQLAlchemy dependency
    Automatically converts markdown tables to SQL
    """

    def __init__(self, db_path: str = "./automated_tables.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.table_registry = {}

        # Create metadata table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS _table_metadata (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            table_name TEXT UNIQUE,
            original_name TEXT,
            num_rows INTEGER,
            num_columns INTEGER,
            column_names TEXT,
            context_text TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        self.conn.commit()

    def markdown_to_dataframe(self, markdown_table: str) -> pd.DataFrame:
        """
        Automatically parse markdown table to DataFrame
        """
        try:
            lines = [l.strip() for l in markdown_table.split('\n') if l.strip()]

            try:
                df = pd.read_csv(
                    StringIO('\n'.join(lines)),
                    sep='|',
                    skipinitialspace=True,
                    skip_blank_lines=True
                )

                df = df.loc[:, ~df.columns.str.match('^Unnamed')]
                df = df.dropna(axis=1, how='all')

                if len(df) > 0:
                    df = df[~df.astype(str).apply(
                        lambda row: row.str.contains('---').any(), axis=1
                    )]

                df.columns = [str(col).strip() for col in df.columns]
                df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)

                return df.reset_index(drop=True)

            except:
                rows = []
                header = None

                for line in lines:
                    cells = [c.strip() for c in line.split('|') if c.strip()]
                    if all('-' in cell or ':' in cell for cell in cells):
                        continue
                    if header is None:
                        header = cells
                    else:
                        rows.append(cells)

                if header and rows:
                    return pd.DataFrame(rows, columns=header)
                return None

        except Exception as e:
            print(f"❌ Parse failed: {e}")
            return None

    def auto_clean_column_names(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Automatically clean column names for SQL compatibility
        """
        new_columns = []
        for col in df.columns:
            clean = re.sub(r'[^a-zA-Z0-9_]', '_', str(col))
            clean = re.sub(r'_+', '_', clean).strip('_').lower()

            if not clean or clean.isdigit():
                clean = f'col_{len(new_columns)}'

            base_clean = clean
            counter = 1
            while clean in new_columns:
                clean = f"{base_clean}_{counter}"
                counter += 1

            new_columns.append(clean)

        df.columns = new_columns
        return df

    def auto_create_sql_table(
        self,
        df: pd.DataFrame,
        table_name: str,
        context: str = ""
    ) -> Tuple[bool, str]:
        """
        Automatically create SQL table from DataFrame
        """
        if df is None or df.empty:
            return False, None

        try:
            df = self.auto_clean_column_names(df)

            safe_table_name = re.sub(r'[^a-zA-Z0-9_]', '_', table_name)
            safe_table_name = safe_table_name[:64].lower()

            base_name = safe_table_name
            counter = 1
            while safe_table_name in self.table_registry:
                safe_table_name = f"{base_name}_{counter}"
                counter += 1

            # Use pandas to_sql
            df.to_sql(
                safe_table_name,
                self.conn,
                if_exists='replace',
                index=False
            )

            self.table_registry[safe_table_name] = {
                'original_name': table_name,
                'rows': len(df),
                'columns': list(df.columns),
                'context': context
            }

            # Insert metadata
            self.cursor.execute("""
            INSERT OR REPLACE INTO _table_metadata
            (table_name, original_name, num_rows, num_columns,
             column_names, context_text)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (
                safe_table_name,
                table_name,
                len(df),
                len(df.columns),
                json.dumps(list(df.columns)),
                context[:1000]
            ))
            self.conn.commit()

            print(f"✅ Created: '{safe_table_name}' ({len(df)} rows, {len(df.columns)} cols)")

            return True, safe_table_name

        except Exception as e:
            print(f"❌ Failed: {e}")
            return False, None

    def get_table_info(self, table_name: str) -> Dict:
        return self.table_registry.get(table_name, {})

    def query_table(self, table_name: str, limit: int = 5) -> pd.DataFrame:
        try:
            return pd.read_sql(
                f"SELECT * FROM {table_name} LIMIT {limit}",
                self.conn
            )
        except Exception as e:
            print(f"Query failed: {e}")
            return None

    def close(self):
        self.conn.close()


# ========================================
# SECTION 5: NODE CREATOR CLASS
# ========================================
class AutomatedNodeCreator:
    """
    Automatically creates nodes from text and table segments
    """

    def __init__(self, sql_creator):
        self.sql_creator = sql_creator

    def create_nodes_from_segments(
        self,
        text_segments: List[Dict],
        table_segments: List[Dict],
        doc_id: str = None
    ) -> List[TextNode]:
        """
        Automatically create all nodes with proper linking
        """
        nodes = []
        doc_id = doc_id or str(uuid.uuid4())

        # Create text nodes
        for i, text_seg in enumerate(text_segments):
            table_refs = self._detect_table_references(
                text_seg['content'],
                table_segments
            )

            node = TextNode(
                text=text_seg['content'],
                id_=f"{doc_id}_text_{i}",
                metadata={
                    'type': 'text',
                    'doc_id': doc_id,
                    'position': text_seg['position'],
                    'has_table_refs': len(table_refs) > 0,
                    'referenced_tables': table_refs,
                    'sql_database': self.sql_creator.db_path
                }
            )
            nodes.append(node)

        # Create table reference nodes
        for i, table_seg in enumerate(table_segments):
            table_info = self.sql_creator.get_table_info(table_seg['name'])

            if table_info:
                description = self._generate_table_description(
                    table_seg['name'],
                    table_info,
                    table_seg['context']
                )

                node = TextNode(
                    text=description,
                    id_=f"{doc_id}_table_{i}",
                    metadata={
                        'type': 'table_reference',
                        'doc_id': doc_id,
                        'sql_table_name': table_seg['name'],
                        'sql_database': self.sql_creator.db_path,
                        'num_rows': table_info['rows'],
                        'num_columns': len(table_info['columns']),
                        'column_names': table_info['columns'],
                        'context': table_seg['context'][:500]
                    }
                )
                nodes.append(node)

        return nodes

    def _detect_table_references(
        self,
        text: str,
        table_segments: List[Dict]
    ) -> List[str]:
        """Auto-detect if text references any tables"""
        refs = []
        text_lower = text.lower()

        for table_seg in table_segments:
            table_name_parts = table_seg['name'].lower().split('_')
            for part in table_name_parts:
                if len(part) > 3 and part in text_lower:
                    refs.append(table_seg['name'])
                    break

        return list(set(refs))

    def _generate_table_description(
        self,
        table_name: str,
        table_info: Dict,
        context: str
    ) -> str:
        """Auto-generate searchable text description for table"""
        description = f"""Table: {table_name}

Context: {context}

This table contains {table_info['rows']} rows of structured data with {len(table_info['columns'])} columns.

Columns: {', '.join(table_info['columns'])}

The data in this table can be queried using SQL from the database: {self.sql_creator.db_path}
Table name for queries: {table_name}
"""
        return description


# ========================================
# SECTION 6: MAIN PIPELINE CLASS
# ========================================
class FullyAutomatedRAGPipeline:
    """
    COMPLETE AUTOMATED PIPELINE
    PDF → Automatic Text/Table Separation → Vector Store + SQL
    """

    def __init__(
        self,
        llamaparse_api_key: str,
        vector_db_path: str = "./auto_vector_db",
        sql_db_path: str = "./auto_tables.db"
    ):
        print("🔧 Initializing pipeline...")

        self.parser = LlamaParse(
            api_key=llamaparse_api_key,
            result_type="markdown",
            verbose=True
        )

        self.content_detector = AutomatedContentDetector()
        self.sql_creator = AutoSQLTableCreatorSimple(sql_db_path)
        self.node_creator = AutomatedNodeCreator(self.sql_creator)

        # Setup FREE embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5"
        )

        # Setup vector store
        self.chroma_client = chromadb.PersistentClient(path=vector_db_path)
        self.collection = self.chroma_client.get_or_create_collection("automated_rag")
        self.vector_store = ChromaVectorStore(chroma_collection=self.collection)
        self.storage_context = StorageContext.from_defaults(vector_store=self.vector_store)

        self.processed_docs = []
        self.all_nodes = []

        print("✅ Pipeline initialized!")

    def process_pdf(self, pdf_path: str) -> Dict:
        """
        MAIN AUTOMATED PIPELINE
        """
        print(f"\n{'='*70}")
        print(f"🚀 AUTOMATED PROCESSING: {pdf_path}")
        print(f"{'='*70}\n")

        # Step 1: Parse PDF
        print("📄 Step 1: Parsing PDF with LlamaParse...")
        documents = self.parser.load_data(pdf_path)
        print(f"   ✅ Parsed {len(documents)} document(s)")

        total_stats = {
            'text_segments': 0,
            'table_segments': 0,
            'nodes_created': 0,
            'sql_tables_created': 0
        }

        # Process each document
        for doc_idx, doc in enumerate(documents):
            print(f"\n📑 Processing Document {doc_idx + 1}/{len(documents)}...")

            # Step 2: Auto detect
            segments = self.content_detector.detect_and_segment(doc.text)
            print(f"   🔍 Text: {segments['stats']['text_count']}, Tables: {segments['stats']['table_count']}")

            # Step 3: Create SQL tables
            print("   🗄️  Creating SQL tables...")
            for table_seg in segments['table_segments']:
                df = self.sql_creator.markdown_to_dataframe(table_seg['content'])

                if df is not None and not df.empty:
                    success, sql_name = self.sql_creator.auto_create_sql_table(
                        df, table_seg['name'], table_seg['context']
                    )

                    if success:
                        table_seg['name'] = sql_name
                        total_stats['sql_tables_created'] += 1

            # Step 4: Create nodes
            print("   🔗 Creating nodes...")
            nodes = self.node_creator.create_nodes_from_segments(
                segments['text_segments'],
                segments['table_segments'],
                doc_id=f"doc_{doc_idx}"
            )

            self.all_nodes.extend(nodes)
            total_stats['nodes_created'] += len(nodes)
            total_stats['text_segments'] += segments['stats']['text_count']
            total_stats['table_segments'] += segments['stats']['table_count']

            print(f"      ✅ Created {len(nodes)} nodes")

        # Step 5: Build vector index
        print(f"\n📊 Step 5: Building vector index...")
        self.index = VectorStoreIndex(
            nodes=self.all_nodes,
            storage_context=self.storage_context,
            show_progress=True
        )

        print(f"\n{'='*70}")
        print(f"✅ PIPELINE COMPLETE!")
        print(f"{'='*70}")
        print(f"📊 Statistics:")
        print(f"   Text segments: {total_stats['text_segments']}")
        print(f"   Tables detected: {total_stats['table_segments']}")
        print(f"   SQL tables created: {total_stats['sql_tables_created']}")
        print(f"   Vector nodes: {total_stats['nodes_created']}")

        return total_stats

    def query(self, question: str, top_k: int = 5) -> Dict:
        """Query the system"""
        print(f"\n{'='*70}")
        print(f"❓ Query: {question}")
        print(f"{'='*70}\n")

        retriever = self.index.as_retriever(similarity_top_k=top_k)
        results = retriever.retrieve(question)

        print(f"📄 Found {len(results)} relevant segments:\n")

        for i, result in enumerate(results):
            print(f"--- Result {i+1} (Score: {result.score:.4f}) ---")
            print(f"Type: {result.metadata['type']}")
            print(f"Content: {result.text[:200]}...\n")

            if result.metadata['type'] == 'table_reference':
                table_name = result.metadata['sql_table_name']
                print(f"🗄️  SQL Table: {table_name}")

                table_data = self.sql_creator.query_table(table_name, limit=5)
                if table_data is not None:
                    print(table_data.to_string())
                print()


# ========================================
# SECTION 7: VERIFICATION CLASS
# ========================================
class AutomatedPipelineVerifier:
    """Verification tools"""

    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.sql_creator = pipeline.sql_creator

    def verify_all_tables(self):
        """Print all SQL tables"""
        print(f"\n{'='*80}")
        print(f"🗄️  SQL TABLES VERIFICATION")
        print(f"{'='*80}\n")

        tables = list(self.sql_creator.table_registry.keys())
        print(f"Total tables: {len(tables)}\n")

        for i, table_name in enumerate(tables):
            print(f"\n{'─'*80}")
            print(f"📊 TABLE {i+1}: {table_name}")
            print(f"{'─'*80}")

            info = self.sql_creator.get_table_info(table_name)

            print(f"\n📋 Metadata:")
            print(f"   Rows: {info['rows']}")
            print(f"   Columns: {info['columns']}")
            print(f"   Context: {info['context'][:200]}...\n")

            print(f"📊 Data Preview:")
            df = self.sql_creator.query_table(table_name, limit=5)
            if df is not None:
                print(df.to_string())

    def list_all_nodes(self):
        """List all vector nodes"""
        nodes = self.pipeline.all_nodes
        text_nodes = [n for n in nodes if n.metadata['type'] == 'text']
        table_nodes = [n for n in nodes if n.metadata['type'] == 'table_reference']

        print(f"\n📦 VECTOR NODES:")
        print(f"   Total: {len(nodes)}")
        print(f"   Text nodes: {len(text_nodes)}")
        print(f"   Table nodes: {len(table_nodes)}")

    def inspect_table(self, table_name: str):
        """Inspect specific table"""
        df = self.sql_creator.query_table(table_name, limit=20)
        print(f"\n📊 TABLE: {table_name}")
        if df is not None:
            print(df.to_string())
        return df


# ========================================
# SECTION 8: USAGE EXAMPLES
# ========================================

print("""
╔════════════════════════════════════════════════════════════════╗
║  AUTOMATED RAG PIPELINE - READY TO USE                         ║
║  Text + Table Extraction with Zero Manual Work                 ║
╚════════════════════════════════════════════════════════════════╝

USAGE:
------
1. Initialize pipeline
2. Process PDF (automatic)
3. Verify results
4. Query the system

See examples below ↓
""")



╔════════════════════════════════════════════════════════════════╗
║  AUTOMATED RAG PIPELINE - READY TO USE                         ║
║  Text + Table Extraction with Zero Manual Work                 ║
╚════════════════════════════════════════════════════════════════╝

USAGE:
------
1. Initialize pipeline
2. Process PDF (automatic)
3. Verify results
4. Query the system

See examples below ↓



In [6]:
# ========================================
# STEP-BY-STEP USAGE
# ========================================

# ===== STEP 1: Set your LlamaParse API key =====
LLAMAPARSE_API_KEY = secret_key  # Get from cloud.llamaindex.ai


# ===== STEP 2: Initialize the pipeline =====
pipeline = FullyAutomatedRAGPipeline(
    llamaparse_api_key=LLAMAPARSE_API_KEY,
    vector_db_path="./ifct_vector_db",
    sql_db_path="./ifct_tables.db"
)


# ===== STEP 3: Process your PDF (AUTOMATIC!) =====
stats = pipeline.process_pdf("/content/IFCT2017.pdf")  # Change path to your PDF


# ===== STEP 4: Verify everything worked =====
verifier = AutomatedPipelineVerifier(pipeline)

# Check all tables
verifier.verify_all_tables()

# Check all nodes
verifier.list_all_nodes()


# ===== STEP 5: Inspect specific table =====
# List available tables first
print("Available tables:", list(pipeline.sql_creator.table_registry.keys()))

# Inspect a specific table
df = verifier.inspect_table("table_proximate_principles")  # Change to your table name


# ===== STEP 6: Query the system =====
pipeline.query("What is the protein content of rice?")
pipeline.query("Show me nutritional information")


# ===== STEP 7: Direct SQL queries (optional) =====
# Query any table directly
rice_data = pipeline.sql_creator.query_table("table_1", limit=10)
print(rice_data)

# Export to CSV
rice_data.to_csv("exported_table.csv", index=False)


🔧 Initializing pipeline...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: Dat

✅ Created: 'table_lakshamana_phalamu__tel__' (47 rows, 4 cols)
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing Document 569/585...
   🔍 Text: 0, Tables: 1
   🗄️  Creating SQL tables...
✅ Created: 'table_luwang__n__' (44 rows, 4 cols)
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing Document 570/585...
   🔍 Text: 0, Tables: 1
   🗄️  Creating SQL tables...
✅ Created: 'table_marutus__kash__' (44 rows, 4 cols)
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing Document 571/585...
   🔍 Text: 0, Tables: 1
   🗄️  Creating SQL tables...
✅ Created: 'table_mooji_hakh__kash__' (45 rows, 4 cols)
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing Document 572/585...
   🔍 Text: 1, Tables: 0
   🗄️  Creating SQL tables...
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing Document 573/585...
   🔍 Text: 0, Tables: 1
   🗄️  Creating SQL tables...
✅ Created: 'table_r003' (45 rows, 3 cols)
   🔗 Creating nodes...
      ✅ Created 1 nodes

📑 Processing D

/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)
/tmp/ipython-input-827554822.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else None)


Generating embeddings:   0%|          | 0/696 [00:00<?, ?it/s]

ValueError: Value for metadata referenced_tables must be one of (str, int, float, None)

In [7]:
import sqlite3

conn = sqlite3.connect("/content/ifct_tables.db")
cursor = conn.cursor()

In [9]:
from pprint import pprint
# List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
pprint(cursor.fetchall())

[('_table_metadata',),
 ('sqlite_sequence',),
 ('table_s__no_',),
 ('delineation_of_food_components_in_published_editio',),
 ('table_components',),
 ('categorization_of_the_states',),
 ('frequency_distribution_of_states',),
 ('number_of_primary_sampling_units__sampling_sites_a',),
 ('analytical_methods_of_nutrient_and_bioactive_compo',),
 ('table_analyte',),
 ('table_code',),
 ('conversion_of_carbohydrate_weights_to_monosacchari',),
 ('jones_factors_for_conversion_of_nitrogen_to_protei',),
 ('metabolizeable_energy_conversion_factors',),
 ('table_a_',),
 ('table_code_1',),
 ('table_code_2',),
 ('table_moisture',),
 ('table_code_3',),
 ('table_moisture_1',),
 ('table_code_4',),
 ('table_q001',),
 ('table_r001',),
 ('table_no__of_regions',),
 ('table_code_5',),
 ('table_no__of_regions_1',),
 ('table_code_6',),
 ('table_no__of_regions_2',),
 ('table_food_code',),
 ('table_no__of_regions_3',),
 ('table_code_7',),
 ('table_no__of_regions_4',),
 ('table_code_8',),
 ('table_code_9',),
 ('table

In [13]:
# View data
table_name = "table_food_code_20"
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
pprint(cursor.fetchall())

[('D074', 'Tomato, green (Solanum lycopersicum)'),
 ('D075', 'Tomato, ripe, hybrid (Solanum lycopersicum)'),
 ('D076', 'Tomato, ripe, local (Solanum lycopersicum)'),
 ('D077', 'Zucchini, green (Cucurbita pepo)'),
 ('D078', 'Zucchini, yellow (Cucurbita pepo)')]


In [14]:
# View data
table_name = "table_food_code_21"
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
pprint(cursor.fetchall())

[('E001', 'Apple, big (Malus domestica)'),
 ('E002', 'Apple, green (Malus domestica)'),
 ('E003', 'Apple, small (Malus domestica)'),
 ('E004', 'Apple, small, Kashmir (Malus domestica)'),
 ('E005', 'Apricot, dried (Prunus armeniaca)')]


In [15]:
# View data
table_name = "table_no__of_regions_34"
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
pprint(cursor.fetchall())

[('6',
  '1.50±0.19',
  None,
  '7.99±0.64',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('6',
  '0.64±0.14',
  None,
  '2.92±0.46',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('6',
  '1.16±0.34',
  None,
  '6.96±1.06',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('1', '1.68', None, '7.90', None, None, None, None, None, None, None, None),
 ('6', None, '0.30±0.07', None, None, None, None, None, None, None, None, None)]


In [ ]:
conn.close()